In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

In [6]:
sys.path.append('../')
from scripts.preprocess import Preprocess
from scripts.logger import Logger
from scripts.plot import Plot
from scripts.utilities import Utilities

preprocess = Preprocess()
plot = Plot()
utils = Utilities()

In [7]:
train_df = utils.get_df(path='data/train.csv',rep='../',rev='tagv1')
test_df = utils.get_df(path='data/test.csv',rep='../', rev='tagv1')
store_df = utils.get_df(path='data/store.csv',rep='../',rev='tagv1')
sub_df = utils.get_df(path='data/sample_submission.csv', rep='../', rev='tagv1')


/home/nardos/Documents/10Academy/Pharmaceutical/Pharmaceutical-Sales-Prediction/notebooks/../scripts/utilities.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(io.StringIO(data))


In [8]:
train_store = pd.read_csv('../data/train_store.csv')
test_store = pd.read_csv('../data/test_store.csv')
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"


In [9]:
numerical_columns = train_store.drop(columns = ['Sales']).select_dtypes(include='number').columns.to_list()
numerical_columns

['Store',
 'DayOfWeek',
 'Customers',
 'Open',
 'Promo',
 'SchoolHoliday',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear']

In [10]:
catagorical_colunms = train_store.select_dtypes(include=['object','boolean']).columns.to_list()[1:]
catagorical_colunms

['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']

In [11]:
Date = train_store.Date

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
oneHotEncoder = OneHotEncoder()


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


In [14]:
catagorical_transformer = Pipeline(steps=[
    ('oneHotEncoder', OneHotEncoder())
])
# catagorical_transformer
numerical_transformers= Pipeline(steps=[
    ('standardScaler', StandardScaler())
])
# numerical_transformers

In [15]:
preproceser = ColumnTransformer(transformers=[
    
    ('numerical_transformers', numerical_transformers, numerical_columns),
    ('catagorical_trandsformer', catagorical_transformer, catagorical_colunms),
   ])

pipeline = Pipeline(steps=[
    ('preproceser', preproceser),
    ('modeling', RandomForestRegressor(n_jobs= 4)),
])

In [16]:
ratio = int(len(train_store) * 0.9)
train_store.sort_values('Date', ascending = True, inplace = True)
# train_store
train, val = train_store.iloc[:ratio, :], train_store.iloc[ratio:, :]
train.shape ,val.shape


((915488, 18), (101721, 18))

In [17]:
x = train.drop(columns = ['Date', 'Sales'])
y = train.Sales.values
pipeline.fit(x,y)

In [ ]:
x_test = val.drop(columns = ['Date', 'Sales'])
y_test = val.Sales.values
prediction = pipeline.predict(x_test)
from sklearn.metrics import mean_squared_error 
mse = mean_squared_error(y_test, prediction)
print(mse)

In [ ]:

mlflow.set_tracking_uri('http://localhost:5000')
        # mlflow.set_tracking_uri('../mlflow_outputs/mlruns')
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
    mlflow.log_model(pipeline, 'Ml_Model')
    mlflow.log_params(pipeline.get_params()['modeling'].get_params())
    mlflow.log_metrics('mean_squared_error', mse)
